# Preamble

This notebook provides an example of how to calculate the performance of a reconfigurable and ISL capable
constellation in terms of the metrics: Age of Information, system response time, and total pass time.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append('../SatLib')
import satbox as sb
import utils as utils


In [3]:
import astropy.units as u
from astropy.time import Time

# Parameters

In [4]:
####### Parameters ####### 
#Determine time to propagate
t2propagate = 3*u.day

#Determine distance Threshold for ISL
distThreshold = 1250 * u.km
####### ####### ####### ####### 

# Initialize ground stations

In [5]:
#BlackSky Ground stations from FCC Report pg 7: https://fcc.report/IBFS/SAT-MOD-20190725-00067/1816346.pdf
latLon = [(13.5, 144.8 ,'Guam'),
          (64.8, -147.5,   'Fairbanks'),
          (78.2, 15.4,   'Svalbard'),
          (50.3, 8.5, 'Usingen'),
          (-46.5, 168.4,  'Invercargill'),
          (42.8, 141.6, 'Chitose'),
          ]

#Create ground stations
groundStations = []
groundStationNodes = []
gsID = 0
for latlon in latLon:
    gs_lat = latlon[0] * u.deg
    gs_lon = latlon[1] * u.deg
    gs_name = latlon[2]
    gs_h = 0 * u.m
    gNode = 'g'+str(gsID)
    g_id = gNode
    groundStationNodes.append(g_id)
    gs = sb.GroundLoc(gs_lon, gs_lat, gs_h, groundID=g_id, identifier=gs_name)
    groundStations.append(gs)
    gsID += 1

# Initialize constellation

In [6]:
t2020 = Time('2019-03-21')

i = 60 * u.deg
t = 12
p = 2
f = 0
alt = 550 * u.km
walker = sb.Constellation.from_walker(i,t,p,f,alt, epoch=t2020)

/Users/manweichan/Desktop/MITPhD/SAS/code/.venv/lib/python3.7/site-packages/astropy/units/decorators.py:253: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


# Initialize a ground target

In [7]:
#Target at 5 deg latitude, 5 deg longitude
lat = 5 * u.deg
lon = 5 * u.deg
gs_h = 0 * u.m #0 meters height of ground target
gsID = 1 #Call this ground station 1

#Initialize ground target
gTarget = sb.GroundLoc(lon, lat, gs_h, groundID=gsID) 

# Run Simulation 

In [8]:
outputReconIsl = utils.get_fastest_downlink_lighting(walker, groundStations, gTarget, t2020, distanceThreshold=distThreshold, t2propagate=t2propagate, simTime=t2propagate,verbose=True)

Step 1 of 5: Generating Schedule
Step 2 of 5: Propagating Satellites
Step 3 of 5: Calculating Relative Data


/Users/manweichan/Desktop/MITPhD/SAS/code/.venv/lib/python3.7/site-packages/astropy/units/quantity.py:486: RuntimeWarning: invalid value encountered in arccos
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


Step 4 of 5: Calculating Access to groundStations
Step 5 of 5: Run Dijkstra


Dijkstra for Sat 5
Dijkstra for Sat 4
Dijkstra for Sat 8
Dijkstra for Sat 7
We found the following best path with a value of 2019-03-21 12:19:45.000.
4 -> g5
We found the following best path with a value of 2019-03-22 12:02:15.000.
4 -> g5
We found the following best path with a value of 2019-03-23 11:45:00.000.
4 -> g5
We found the following best path with a value of 2019-03-21 11:59:45.000.
8 -> 5 -> g5
We found the following best path with a value of 2019-03-22 13:53:00.000.
8 -> g1
We found the following best path with a value of 2019-03-23 11:25:00.000.
8 -> 0 -> g5


In [9]:
#Extract raw data outputs
lightReconIsl = outputReconIsl["outputLighting"]

In [10]:
#Calculate metrics
metricReconIslLighting = utils.calc_metrics(lightReconIsl, T=t2propagate)

In [13]:
metricReconIslLighting

{'AoI': <Quantity 620.60812355 min>,
 'srt': <Quantity 719.75 min>,
 'passTimeSum': <Quantity 330. s>}